In [3]:
import numpy as np
import random, copy, os, math
import operator
from scipy import sparse
import time

In [3]:
%run dependent_rounding.ipynb

In [4]:
def set_unique_ids(objects):
    for idx, obj in enumerate(objects):
        obj.set_unique_id(idx)

# Objectives for LP

In [39]:
def get_operator_objective_kiid_sk(profit_matrix):
    c = -1 * profit_matrix.flatten()
    return c


def get_driver_objective_kiid_sk(utility):
    c = np.zeros([utility.shape[0] * utility.shape[1] + 1])
    c[c.shape[0] - 1] = -1
    return c

def get_rider_objective_kiid_sk(utility):
    c = np.zeros([utility.shape[0] * utility.shape[1] + 1])
    c[c.shape[0] - 1] = -1
    return c


# Constraints for LP

In [1]:
# LP related functions for scipy solver
def get_inequalities_operator_kiid_sk(drivers,requests, prob):
    num_drivers = len(drivers)
    num_types = len(requests)

    A, b = [], [] # model all inequalities as A * x <= b

    temp_arr = sparse.lil_array(np.zeros([num_drivers,num_types]))
    sparse_prob = sparse.lil_array(prob)

    #(4.b)
    for driver in drivers:
        driver_idx = driver.u_id
        temp_arr[driver_idx,:] = sparse_prob[[driver_idx],:]

        A.append(temp_arr.reshape(1,num_drivers * num_types))
        b.append(1.)
        temp_arr[driver_idx,:] = 0

    #(4.c)
    for driver in drivers:
        driver_idx = driver.u_id
        temp_arr[driver_idx,:] = 1

        A.append(temp_arr.reshape(1,num_drivers * num_types))
        b.append(driver.quota)
        temp_arr[driver_idx,:] = 0

    #(4.d)
    for request in requests:
        request_idx = request.u_id
        temp_arr[:,request_idx] = sparse_prob[:,[request_idx]]

        A.append(temp_arr.reshape(1,num_drivers * num_types))
        b.append(1.)
        temp_arr[:,request_idx] = 0

    #(4.e)
    for request in requests:
        request_idx = request.u_id
        temp_arr[:,request_idx] = 1
        A.append(temp_arr.reshape(1,num_drivers * num_types))
        b.append(request.individual_tolerance)
        temp_arr[:,request_idx] = 0

    return sparse.vstack(A), np.array(b)

def get_inequalities_driver_kiid_sk(drivers, requests, prob, utility):
    num_drivers = len(drivers)
    num_types = len(requests)

    A, b = [], [] # model all inequalities as A * x <= b

    temp_arr = sparse.lil_array(np.zeros([num_drivers,num_types]))
    sparse_prob = sparse.lil_array(prob)

    #(4.b)
    for driver in drivers:
        driver_idx = driver.u_id
        temp_arr[driver_idx,:] = sparse_prob[[driver_idx],:]

        A.append(temp_arr.reshape(1,num_drivers * num_types))
        b.append(1.)
        temp_arr[driver_idx,:] = 0

    #(4.c)
    for driver in drivers:
        driver_idx = driver.u_id
        temp_arr[driver_idx,:] = 1

        A.append(temp_arr.reshape(1,num_drivers * num_types))
        b.append(driver.quota)
        temp_arr[driver_idx,:] = 0

    #(4.d)
    for request in requests:
        request_idx = request.u_id
        temp_arr[:,request_idx] = sparse_prob[:,[request_idx]]

        A.append(temp_arr.reshape(1,num_drivers * num_types))
        b.append(1.)
        temp_arr[:,request_idx] = 0

    #(4.e)
    for request in requests:
        request_idx = request.u_id
        temp_arr[:,request_idx] = 1
        A.append(temp_arr.reshape(1,num_drivers * num_types))
        b.append(request.individual_tolerance)
        temp_arr[:,request_idx] = 0

    A = sparse.vstack(A)
    additional_vec = sparse.lil_array(np.zeros([A.shape[0],1]))
    A = sparse.hstack((A,additional_vec))
    b = np.array(b)

    c = -1 * utility * prob

    race_0_mask = np.zeros([num_drivers,1])
    for driver in drivers:
        if driver.race == 'white':
            race_0_mask[driver.u_id,0] = 1

    num_race_0 = np.count_nonzero(race_0_mask)
    num_race_1 = num_drivers - num_race_0

    race_0_mask = np.tile(race_0_mask,(1,num_types))
    race_1_mask = 1 - race_0_mask

    util_race_0 = race_0_mask * c
    util_race_0 = util_race_0.flatten()
    util_race_1 = race_1_mask * c
    util_race_1 = util_race_1.flatten()

    if num_race_0 == 0:
        mean_util_race_1 =  util_race_1 / num_race_1

        eta1 = list(mean_util_race_1)
        eta1.append(1)
        eta1 = np.array(eta1)
        eta1 = np.reshape(eta1,[1,eta1.shape[0]])

        eta1 = sparse.lil_array(eta1)

        A = sparse.vstack((A,eta1))
        b = list(b)
        b.append(0)
        b = np.array(b)

    elif num_race_1 == 0:
        mean_util_race_0 =  util_race_0 / num_race_0

        eta0 = list(mean_util_race_0)
        eta0.append(1)
        eta0 = np.array(eta0)
        eta0 = np.reshape(eta0,[1,eta0.shape[0]])

        eta0 = sparse.lil_array(eta0)

        A = sparse.vstack((A,eta0))
        b = list(b)
        b.append(0)
        b = np.array(b)

    else:
        mean_util_race_0 =  util_race_0 / num_race_0
        mean_util_race_1 =  util_race_1 / num_race_1

        eta0 = list(mean_util_race_0)
        eta0.append(1)
        eta0 = np.array(eta0)
        eta0 = np.reshape(eta0,[1,eta0.shape[0]])

        eta1 = list(mean_util_race_1)
        eta1.append(1)
        eta1 = np.array(eta1)
        eta1 = np.reshape(eta1,[1,eta1.shape[0]])

        eta0 = sparse.lil_array(eta0)
        eta1 = sparse.lil_array(eta1)

        A = sparse.vstack((A,eta0))
        A = sparse.vstack((A,eta1))

        b = list(b)
        b.append(0)
        b.append(0)
        b = np.array(b)

    bounds = []
    for i in range(num_drivers * num_types):
        bounds.append([0,1])

    bounds.append([None,None])

    return A, b, bounds


def get_inequalities_rider_kiid_sk(drivers, requests, prob, utility):
    num_drivers = len(drivers)
    num_types = len(requests)

    A, b = [], [] # model all inequalities as A * x <= b

    temp_arr = sparse.lil_array(np.zeros([num_drivers,num_types]))
    sparse_prob = sparse.lil_array(prob)

    #(4.b)
    for driver in drivers:
        driver_idx = driver.u_id
        temp_arr[driver_idx,:] = sparse_prob[[driver_idx],:]

        A.append(temp_arr.reshape(1,num_drivers * num_types))
        b.append(1.)
        temp_arr[driver_idx,:] = 0

    #(4.c)
    for driver in drivers:
        driver_idx = driver.u_id
        temp_arr[driver_idx,:] = 1

        A.append(temp_arr.reshape(1,num_drivers * num_types))
        b.append(driver.quota)
        temp_arr[driver_idx,:] = 0

    #(4.d)
    for request in requests:
        request_idx = request.u_id
        temp_arr[:,request_idx] = sparse_prob[:,[request_idx]]

        A.append(temp_arr.reshape(1,num_drivers * num_types))
        b.append(1.)
        temp_arr[:,request_idx] = 0

    #(4.e)
    for request in requests:
        request_idx = request.u_id
        temp_arr[:,request_idx] = 1
        A.append(temp_arr.reshape(1,num_drivers * num_types))
        b.append(request.individual_tolerance)
        temp_arr[:,request_idx] = 0

    A = sparse.vstack(A)
    additional_vec = sparse.lil_array(np.zeros([A.shape[0],1]))
    A = sparse.hstack((A,additional_vec))
    b = np.array(b)

    c = -1 * utility * prob

    race_0_mask = np.zeros([1,num_types])
    for rider in requests:
        if rider.race == 'white':
            race_0_mask[0,rider.u_id] = 1

    num_race_0 = np.count_nonzero(race_0_mask)
    num_race_1 = num_types - num_race_0

    race_0_mask = np.tile(race_0_mask,(num_drivers,1))
    race_1_mask = 1 - race_0_mask

    util_race_0 = race_0_mask * c
    util_race_0 = util_race_0.flatten()
    util_race_1 = race_1_mask * c
    util_race_1 = util_race_1.flatten()

    if num_race_0 == 0:
        mean_util_race_1 =  util_race_1 / num_race_1

        eta1 = list(mean_util_race_1)
        eta1.append(1)
        eta1 = np.array(eta1)
        eta1 = np.reshape(eta1,[1,eta1.shape[0]])

        eta1 = sparse.lil_array(eta1)

        A = sparse.vstack((A,eta1))
        b = list(b)
        b.append(0)
        b = np.array(b)

    elif num_race_1 == 0:
        mean_util_race_0 =  util_race_0 / num_race_0

        eta0 = list(mean_util_race_0)
        eta0.append(1)
        eta0 = np.array(eta0)
        eta0 = np.reshape(eta0,[1,eta0.shape[0]])

        eta0 = sparse.lil_array(eta0)

        A = sparse.vstack((A,eta0))
        b = list(b)
        b.append(0)
        b = np.array(b)

    else:
        mean_util_race_0 =  util_race_0 / num_race_0
        mean_util_race_1 =  util_race_1 / num_race_1

        eta0 = list(mean_util_race_0)
        eta0.append(1)
        eta0 = np.array(eta0)
        eta0 = np.reshape(eta0,[1,eta0.shape[0]])

        eta1 = list(mean_util_race_1)
        eta1.append(1)
        eta1 = np.array(eta1)
        eta1 = np.reshape(eta1,[1,eta1.shape[0]])

        eta0 = sparse.lil_array(eta0)
        eta1 = sparse.lil_array(eta1)

        A = sparse.vstack((A,eta0))
        A = sparse.vstack((A,eta1))

        b = list(b)
        b.append(0)
        b.append(0)
        b = np.array(b)

    bounds = []
    for i in range(num_drivers * num_types):
        bounds.append([0,1])

    bounds.append([None,None])

    return A, b, bounds


# Functions to calculate Fairness and Profit

In [40]:
def get_profit(drivers_match,requests_match,operator_utility):
    #Computes profit
    len_drivers_match = len(drivers_match)
    len_requests_match = len(requests_match)

    profit = 0

    assert len_drivers_match == len_requests_match
    for i in range(len_drivers_match):
        if (drivers_match[i] is not None) and (requests_match[i] is not None): #it is a match
            driver_idx = drivers_match[i].u_id
            request_idx = requests_match[i].u_id
            profit += operator_utility[driver_idx,request_idx]

    #returns the utilities of each driver and rider
    return profit


def get_utils(drivers_match,requests_match,num_drivers,num_requests,driver_utility_mat,rider_utility_mat):
    #Computes utilities of drivers and riders
    len_drivers_match = len(drivers_match)
    len_requests_match = len(requests_match)

    driver_utils = np.zeros(num_drivers)
    rider_utils = np.zeros(num_requests)

    assert len_drivers_match == len_requests_match
    for i in range(len_drivers_match):
        if (drivers_match[i] is not None) and (requests_match[i] is not None): #it is a match
            driver_idx = drivers_match[i].u_id
            request_idx = requests_match[i].u_id
            driver_utils[driver_idx] = driver_utility_mat[driver_idx,request_idx]
            rider_utils[request_idx] = rider_utility_mat[driver_idx,request_idx]

    #returns the utilities of each driver and rider
    return driver_utils, rider_utils

def util_to_fairness(drivers,requests,d_utils,r_utils):
    #compute the fairness objectives given utilities of drivers and riders

    #FOR DRIVER:
    #creating a dictionary of all races
    race_to_idx = {}
    num_races = []
    sum_util_by_race = []
    for d in drivers:
        if d.race not in race_to_idx:
            race_to_idx[d.race] = len(race_to_idx)
            num_races.append(1)
            sum_util_by_race.append(d_utils[d.u_id])
        else:
            num_races[race_to_idx[d.race]] += 1
            sum_util_by_race[race_to_idx[d.race]] += d_utils[d.u_id]

    mean_utils_by_race = np.array(sum_util_by_race) / np.array(num_races)
    d_fairness = np.min(mean_utils_by_race)

    #FOR RIDER:
    #creating a dictionary of all races
    race_to_idx = {}
    num_races = []
    sum_util_by_race = []
    for r in requests:
        if r.race not in race_to_idx:
            race_to_idx[r.race] = len(race_to_idx)
            num_races.append(1)
            sum_util_by_race.append(r_utils[r.u_id])
        else:
            num_races[race_to_idx[r.race]] += 1
            sum_util_by_race[race_to_idx[r.race]] += r_utils[r.u_id]

    mean_utils_by_race = np.array(sum_util_by_race) / np.array(num_races)
    r_fairness = np.min(mean_utils_by_race)
    return d_fairness,r_fairness


In [43]:
def driver_acceptance(driver, request, probability_matrix):
    #determine whether the driver will be matched, rejected, or leave the graph

    p_f = probability_matrix[driver.u_id, request.u_id]
    assert p_f != -1
    if driver.is_present:
        decision = np.random.choice(np.arange(2), size=1, p=[p_f, 1-p_f])[0]
        if decision == 0: #driver accepted the trip
            driver.capacity -= 1
            if driver.capacity == 0:
                driver.is_present = False
            return driver
        else:
            driver.quota -= 1
            if driver.quota <= 0:
                # make the driver unavailable
                driver.is_present = False
                #print(driver.u_id, ' out')
            return None
    else:
        print('trying to match not present driver')
        return None


def dependent_rounding_subroutine_express(x_r, probability_matrix, drivers, request):
    # x_r is a vector of all drivers linked to a request
    # use x_r to construct X_r
    X_r = dependent_rounding(x_r)
    presence_mask = np.ones(len(drivers))
    for driver in drivers:
        if not driver.is_present:
            presence_mask[driver.u_id] = 0

    X_r = X_r * presence_mask

    possible_drivers = np.where(X_r == 1)[0]

    for i in range(min(request.individual_tolerance, len(possible_drivers))):
        matched_driver = driver_acceptance(drivers[possible_drivers[i]], request, probability_matrix)
        if matched_driver is not None:
            return matched_driver, X_r

    return None, None

def INAdap_express(alpha, beta, gamma, request, drivers_copy, probability_matrix, x_operator, x_driver, x_rider):
    # There are 2 possible actions, choose assignment based on x_f* (profitable) and y_f* (fair) or reject

    #first decide a meta action, whether to assign or reject
    action = np.random.choice([0,1],p = [alpha + beta + gamma, 1-alpha - beta-gamma])

    if action == 0: # choose to assign

        possible_zs = [x_operator[:,request.u_id], x_driver[:,request.u_id], x_rider[:,request.u_id]]
        z = np.random.choice([0, 1, 2], p=[alpha, beta, gamma])

        #use dependent rounding to pick driver, the driver is non if no match is made
        dr_sub = dependent_rounding_subroutine_express(possible_zs[z], probability_matrix,
                                             drivers_copy, request)
        return dr_sub[0]
    else: # reject the request
        assert alpha + beta + gamma < 1, "If alpha + beta + gamma == 1, this should not happen"
        return None

def run_TSGF_express(loop_idx,record_book, all_requests, drivers_copy, probability_matrix, x_operator, x_driver,
                  x_rider, alpha, beta, gamma):

    for j in range(len(all_requests)):
        r = random.choice(all_requests)

        #see which driver the algorithm matches, None if no match found
        matched_driver = INAdap_express(alpha, beta, gamma, r, drivers_copy, probability_matrix,
                                x_operator, x_driver, x_rider)

        #if a match actually happened
        if (matched_driver != None):
            record_book[loop_idx,matched_driver.u_id,r.u_id] += 1


# Greedy Algorithms

In [44]:

def run_greedy_o(all_requests, drivers_copy, probability_matrix, operator_utility):
    #NOTE: THIS ALGORITHM ONLY WORKS WITH TWO DRIVER GROUPS
    matches, profit = [], 0

    probs = probability_matrix * operator_utility

    available_drivers_idxs = [i for i in range(len(drivers_copy))]

    requests_ordered = []
    for i in range(len(all_requests)):
        request = random.choice(all_requests)
        requests_ordered.append(copy.deepcopy(request))
        driver_assigned = False
        for probing in range(request.individual_tolerance): #probing at most individual tolerance many times

            avail_utilities = []
            for driver_idx in available_drivers_idxs:
                avail_utilities.append(probs[drivers_copy[driver_idx].u_id, request.u_id])

            if len(available_drivers_idxs) == 0:
                driver_assigned = False
                break

            avail_utilities = np.array(avail_utilities)

            candidate_idx = available_drivers_idxs[np.argmax(avail_utilities)] #the index of the index in focus group
            assert drivers_copy[candidate_idx].is_present
            assigned_driver = driver_acceptance(drivers_copy[candidate_idx], request, probability_matrix)

            if assigned_driver is not None:
                driver_assigned = True
                matches.append(assigned_driver)
                profit += operator_utility[assigned_driver.u_id,request.u_id]
                available_drivers_idxs.remove(candidate_idx)

                break

            elif not drivers_copy[candidate_idx].is_present:
                available_drivers_idxs.remove(candidate_idx)

        if not driver_assigned:
            matches.append(None)

    assert len(matches) == len(all_requests)

    return matches, requests_ordered

def run_greedy_r(all_requests, drivers_copy, probability_matrix, rider_utility):
    #NOTE: THIS ALGORITHM ONLY WORKS WITH TWO DRIVER GROUPS
    matches, profit = [], 0
    probs = probability_matrix * rider_utility
    available_drivers_idxs = [i for i in range(len(drivers_copy))]

    requests_ordered = []
    for i in range(len(all_requests)):
        request = random.choice(all_requests)
        requests_ordered.append(copy.deepcopy(request))
        driver_assigned = False
        for probing in range(request.individual_tolerance): #probing at most individual tolerance many times
            avail_utilities = []
            for driver_idx in available_drivers_idxs:
                avail_utilities.append(probs[drivers_copy[driver_idx].u_id, request.u_id])

            if len(available_drivers_idxs) == 0:
                driver_assigned = False
                break

            avail_utilities = np.array(avail_utilities)
            candidate_idx = available_drivers_idxs[np.argmax(avail_utilities)] #the index of the index in focus group
            assert drivers_copy[candidate_idx].is_present
            assigned_driver = driver_acceptance(drivers_copy[candidate_idx], request, probability_matrix)

            if assigned_driver is not None:
                driver_assigned = True
                matches.append(assigned_driver)
                available_drivers_idxs.remove(candidate_idx)

                break

            elif not drivers_copy[candidate_idx].is_present:
                available_drivers_idxs.remove(candidate_idx)

        if not driver_assigned:
            matches.append(None)

    assert len(matches) == len(all_requests)
    # print(len(matches), len(all_requests))
    # assert len(matches) == len(all_requests)
    # print (f'{np.sum([d.is_present for d in drivers_copy])} unmatched drivers, \
    #     {np.sum([d.capacity if d.is_present else 0 for d in drivers_copy])} empty seats, \
    #     {np.sum([r is None for r in matches])} unmatched requests, \
    #     {len(set(all_requests)) - len(set([all_requests[x] for x in range(len(matches)) if matches[x] is not None]))} unmatched request types')
    return matches, requests_ordered

def run_greedy_d(all_requests, drivers_copy, probability_matrix, driver_utility):
    #NOTE: THIS ALGORITHM ONLY WORKS WITH TWO DRIVER GROUPS
    matches, profit = [], 0

    drivers_by_group = [[],[]]

    probs = probability_matrix * driver_utility

    utils = [0,0]

    for i in range(len(drivers_copy)):
        if drivers_copy[i].race == 'black':
            drivers_by_group[0].append(i)
        else:
            drivers_by_group[1].append(i)

    len_groups = [len(drivers_by_group[0]),len(drivers_by_group[1])]

    assert len(drivers_by_group[0]) + len(drivers_by_group[1]) == len(drivers_copy)

    requests_ordered = []

    for i in range(len(all_requests)):
        request = random.choice(all_requests)
        requests_ordered.append(copy.deepcopy(request))
        driver_assigned = False
        for probing in range(request.individual_tolerance): #probing at most individual tolerance many times

            #print(request.individual_tolerance)
            mean_util0 = utils[0] / len_groups[0]
            mean_util1 = utils[1] / len_groups[1]

            focus_group_idx = np.argmin(np.array([mean_util0,mean_util1]))

            #print(mean_util0, mean_util1, focus_group_idx)

            if len(drivers_by_group[focus_group_idx]) == 0 and len(drivers_by_group[1 - focus_group_idx]) > 0:
                focus_group_idx = 1 - focus_group_idx
            elif len(drivers_by_group[focus_group_idx]) == 0 and len(drivers_by_group[1 - focus_group_idx]) == 0:
                focus_group_idx = None #drivers depleted
                driver_assigned = False
                break

            focus_group = drivers_by_group[focus_group_idx]

            avail_utilities = []
            for driver_idx in focus_group:
                avail_utilities.append(probs[drivers_copy[driver_idx].u_id, request.u_id])

            avail_utilities = np.array(avail_utilities)

            candidate_idx = focus_group[np.argmax(avail_utilities)] #the index of the index in focus group
            assert drivers_copy[candidate_idx].is_present
            assigned_driver = driver_acceptance(drivers_copy[candidate_idx], request, probability_matrix)

            if assigned_driver is not None:
                driver_assigned = True
                matches.append(assigned_driver)
                drivers_by_group[focus_group_idx].remove(candidate_idx)
                utils[focus_group_idx] += driver_utility[assigned_driver.u_id,request.u_id]

                break

            elif not drivers_copy[candidate_idx].is_present:
                drivers_by_group[focus_group_idx].remove(candidate_idx)

        if not driver_assigned:
            matches.append(None)

    assert len(matches) == len(all_requests)
    # print(len(matches), len(all_requests))
    # assert len(matches) == len(all_requests)
    # print (f'{np.sum([d.is_present for d in drivers_copy])} unmatched drivers, \
    #     {np.sum([d.capacity if d.is_present else 0 for d in drivers_copy])} empty seats, \
    #     {np.sum([r is None for r in matches])} unmatched requests, \
    #     {len(set(all_requests)) - len(set([all_requests[x] for x in range(len(matches)) if matches[x] is not None]))} unmatched request types')
    return matches, requests_ordered
